In [13]:
import ee
import numpy as np
import pandas as pd

from earthshot import mon_stats
from earthshot import water_viz as vis
from earthshot import normalize as norm
from statistics import mean

import folium
from folium import plugins

import matplotlib.pyplot as plt
%matplotlib inline


In [14]:
ee.Initialize()

In [15]:
#Import Global MAR Inventory
MAR_swales = ee.FeatureCollection('users/amgadellaboudy/Global_MAR_Inventory')
MAR_swales = MAR_swales.remap(['In-Channel Modification', 
                  'Induced Bank Filtration', 
                  'Rainwater and Run-off Harvesting', 
                  'Spreading Methods', 
                  'Well, Shaft and Borehole Recharge'],
                [0,1,2,3,4],
                'main_mar_t')

In [16]:
#Import variables in (Slope, Porosity, Runoff, Soil Types, Annual Precipitation, water accumulation)
slope_img = ee.Image('users/jamesmcc/merit_slope/merit_terrain_slope')

smap_usda_clim = ee.ImageCollection('users/jamesmcc/smap_usda_climatology')
avail_porosity = (smap_usda_clim
                  .filter(ee.Filter.eq('band', 'avail_porosity_mm')))
avail_porosity_img = avail_porosity.sum()

runoff_clim = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('surface_runoff')
runoff_clim_m = mon_stats.bands_avgs(['surface_runoff'], runoff_clim)
runoff_img = ee.ImageCollection(runoff_clim_m['avgs'].get('surface_runoff')).sum().multiply(720)

precip_clim = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01")
precip_img = precip_clim.reduce('sum')

soil_types = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")

top_soils = soil_types.expression('soil_0 + soil_10 + soil_30',
                             {'soil_0': soil_types.select('b0'),
                             'soil_10': soil_types.select('b10'),
                             'soil_30': soil_types.select('b30')})

bottom_soils = soil_types.expression('soil_60 + soil_100 + soil_200',
                                 {'soil_60': soil_types.select('b60'),
                                  'soil_100': soil_types.select('b100'),
                                  'soil_200': soil_types.select('b200')})

water_img = ee.Image('WWF/HydroSHEDS/15ACC').select('b1')

In [17]:
#Collect images into one image, convert into feature collection that includes location and type of MAR interventions

img_1 = slope_img.addBands(avail_porosity_img)
img_2 = img_1.addBands(runoff_img)
img_3 = img_2.addBands(precip_img)
img_4 = img_3.addBands(top_soils)
img_5 = img_4.addBands(bottom_soils)
training_img = img_5.addBands(water_img)

target = 'main_mar_t'
bands = ['slope', 'avail_porosity_mm', 'surface_runoff', 'sum', 'b0', 'b60', 'b1']

training = training_img.sampleRegions(collection= MAR_swales, properties= [target], scale= 30)


In [18]:
#Test/ validation split, fit classifier for training data

split = 0.8
training = training.randomColumn()

training_split = training.filter(ee.Filter.lt('random', split));
validation_split = training.filter(ee.Filter.gte('random', split));

classifier = ee.Classifier.smileGradientTreeBoost(1000).train(training_split, target, bands)

In [19]:
#Create predicted MAR intervention image and validation data

classified = training_img.select(bands).classify(classifier)
validated = validation_split.classify(classifier)

In [20]:
#Training accuracy

trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

Resubstitution error matrix:  [[148, 0, 0, 1, 0], [0, 101, 0, 0, 2], [0, 0, 45, 0, 1], [2, 0, 0, 194, 2], [0, 0, 0, 0, 231]]
Training overall accuracy:  0.9889958734525447


In [21]:
#Validation accuracy

validationAccuracy = validated.errorMatrix('main_mar_t', 'classification')
print('Validation error matrix: ', validationAccuracy.getInfo())
print('Validation overall accuracy: ', validationAccuracy.accuracy().getInfo())

Validation error matrix:  [[22, 0, 0, 3, 3], [0, 17, 0, 3, 2], [1, 1, 2, 0, 2], [3, 5, 1, 24, 14], [2, 2, 4, 10, 37]]
Validation overall accuracy:  0.6455696202531646


In [22]:
#Create map of predicted successful MAR projects based on variables

import geemap.eefolium as geemap

the_map = geemap.Map()

labels = ['In-Channel Modification', 
                  'Induced Bank Filtration', 
                  'Rainwater and Run-off Harvesting', 
                  'Spreading Methods', 
                  'Well, Shaft and Borehole Recharge']

colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']


vis_params = {
    'min': 0, 'max': 4, 'dimensions': 512,
    'palette': ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']}

the_map.addLayer(classified, vis_params, name = 'MAR techniques')
the_map.add_legend(title='Legend', labels=labels, colors=colors)
vis.folium_display(the_map)

In [23]:
#Export to GEE
#xx = ee.batch.Export.image.toAsset(
        #classified, 
        #assetId = 'users/amgadellaboudy/Global_MAR_Classification',
       # maxPixels = 100000000000).start()